In [31]:
import time
import numpy as np
import pandas as pd
import datetime

from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Controlling the webdriver  
Firstly, we want to access the webpage doctolib. To do that we have to create a connector with Selenium and the launch the driver

In [32]:
tempsAttente = 3    #temps d'attente en secondes avant qu'il y a un timeout de Webdriverwait.until()

class Driver:

    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--disable-blink-features")
        options.add_argument("disable-popup-blocking")
        options.add_argument("disable-notifications")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument("--incognito")
        # prefs = {"profile.default_content_settings.geolocation" : "2"}
        # options.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome(ChromeDriverManager().install())


    #Lancer google chrome avec la page doctolib

    def connect(self, url):
        self.driver.get(url)
        # TabsInteraction.TabsCreation(self.driver, "https://www.doctolib.fr", "")
        # TabsInteraction.switchToTab(self.driver, 0)
        print("Connecting to", url)
        time.sleep(1)


    #for now we get only the first doctors that appear in the page
    def getDoctors(self, specialization, city):

        doctors = []

        try :
            #writing the specialization
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(specialization)
            time.sleep(1)

            #writing the city
            WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]')))
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').clear()
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(city)
            time.sleep(1)
            #We need to enter the city to register it
            self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(Keys.ENTER)

            # WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]')))
            # self.driver.find_element(By.XPATH, '//span[@class="searchbar-submit-button-label dl-text-transform-none"]').click()

            WebDriverWait(self.driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//a[text()='Suivant']")))
            self.driver.find_element(By.XPATH, "//a[text()='Suivant']").click()

            print('Searching results for :', specialization, 'in', city, 'at', datetime.datetime.now())
            time.sleep(2)

            WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]')))
            #number of doctors in city
            nbr_result = self.driver.find_element(By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]').text.split(" ")[0]
            #nbr of doctors we want
            nbr = int(int(nbr_result)*0.2)
            #nbr of pages
            pages = nbr//20
            # print(pages)

            for _ in range(pages + 1):
                
                self.driver.find_element(By.CLASS_NAME, "")
                self.driver.execute_script("window.scrollTo(0, 6300);")
                
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-search-result"]')))
                docs = self.driver.find_elements(By.XPATH, '//div[@class="dl-search-result"]')
                
                for doctor in docs:
                    #first we get the "easy" features
                    name = doctor.find_element(By.XPATH, './/h3[@data-test="dl-search-result-name"]').text
                    city = city
                    specialization = specialization

                    #the the harder ones
                    try:
                        WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]')))
                        first_dispo = doctor.find_element(By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]').get_attribute('aria-label')
                    except:
                        try:
                            WebDriverWait(self.driver, 1).until(EC.visibility_of_element_located((By.XPATH, './/span[@class="dl-button-label"]')))
                            first_dispo = doctor.find_elements(By.XPATH, './/span[@class="dl-button-label"]')[1].text.split(" ")[3:5]
                        except:
                            try:
                                WebDriverWait(self.driver, 1).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150"]')))
                                first_dispo = 'pas de disponibilité'
                            except:
                                first_dispo = 'null'    #means that we didn't manage to find the date
                            

                    print(name, first_dispo)
                    fee = ""
                    doctors.append(Doctor(name, city, specialization, first_dispo, fee))

                try:
                    from selenium.webdriver import ActionChains

                    next_button = self.driver.wait.until(EC.visibility_of_element_located((By.XPATH, "//a[text()='Suivant']")))
                    actions = ActionChains(self.driver)
                    actions.move_to_element(next_button).perform()
                    next_button.click()
                    WebDriverWait(self.driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//a[text()='Suivant']")))
                    self.driver.find_element(By.XPATH, "//a[text()='Suivant']").click()
                except:
                    pass
        
        except :
            print('Could not get the doctors for ' + specialization + ' in ' + city)

        return doctors


    def getFees(self, name):
        self.driver.find_element(By.XPATH, '//div[@class="logo-doctolib logo-doctolib-white"]').click()

        #writing the doctor name
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(name)
        #We need to enter the city to register it
        #TODO : add the city
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]').send_keys(Keys.ENTER)

        try:
            WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-fee-tag"]')))
            fee = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-fee-tag"]').text
            fee = str(int(fee.split(" ")[0]))
        except:
            try :
                WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-text"]//div[1]')))
                first_sub_div = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-text"]//div[1]')
                fee = first_sub_div.text
            except:
                fee = 'null'        #if the fee tag cannot be found it means that it is conventionné

        return fee
            
        


We created a Doctor class to manipulate more easily the doctors features

In [33]:
class Doctor():
    def __init__(self, name, city, specialization, first_dispo, fee):
        self.name = name
        self.city = city
        self.specialization = specialization
        self.first_dispo = first_dispo
        self.fee = fee

    def features(self):
        return(self.name, self.city, self.specialization, self.first_dispo, self.fee)

These are the cities and specializations we looked at first :  
Lyon and Paris are bug cities and Sancerre and Confolens are small ones, we did this in order to compare fees and disponibilites at first glance.

In [34]:
# cities = ['Poitiers', 'Rochefourchat', 'Majastres', 'Aulan', 'Rouen', 'Lorient', 'Niort', 'Valence', 'Quimper', 'Paris', 'Lyon', 'Marseille', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Poitiers', 'Dunkerque', 'Pau', 'Mulhouse', 'Lorient', 'Le Havre', 'Annecy', 'Troyes', 'La Rochelle', 'Amiens']
# specializations = ['Medecin generaliste', 'Chirurgien-dentiste', 'Ophtalmologue', 'Dermatologue et venerologue', 'Cardiologue', 'Rhumatologue']

cities = ['Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Poitiers', 'Dunkerque', 'Pau', 'Mulhouse', 'Lorient', 'Le Havre', 'Annecy', 'Troyes', 'La Rochelle', 'Amiens']
# specializations = ['Medecin generaliste', 'Chirurgien-dentiste', 'Ophtalmologue', 'Dermatologue et venerologue', 'Cardiologue', 'Rhumatologue']
specializations = ['medecin-generaliste', 'dentiste', 'ophtalmologue', 'dermatologue', 'cardiologue', 'rhumatologue']


In [35]:
connector = Driver()
connector.connect("https://www.doctolib.fr")
doctors = []

#Get all the doctors names, cities, specializations and their next possible reservation

for city in cities:
    for spec in specializations:
        doctors = doctors + connector.getDoctors(spec, city)
        docs = []
        for doc in doctors:
            docs.append(doc.features())
        pd.DataFrame(docs, columns=['Name', 'City', 'Specialization', 'Waiting Time', 'Fees']).to_csv("doctors_new.csv")

/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_3132/2052145444.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr
Could not get the doctors for Medecin generaliste in Toulouse
Could not get the doctors for Chirurgien-dentiste in Toulouse
Could not get the doctors for Ophtalmologue in Toulouse
Could not get the doctors for Dermatologue et venerologue in Toulouse
Could not get the doctors for Cardiologue in Toulouse
Could not get the doctors for Rhumatologue in Toulouse
Could not get the doctors for Medecin generaliste in Nice
Could not get the doctors for Chirurgien-dentiste in Nice
Could not get the doctors for Ophtalmologue in Nice
Could not get the doctors for Dermatologue et venerologue in Nice
Could not get the doctors for Cardiologue in Nice
Could not get the doctors for Rhumatologue in Nice
Could not get the doctors for Medecin generaliste in Nantes
Could not get the doctors for Chirurgien-dentiste in Nantes
Could not get the doctors for Ophtalmologue in Nantes
Could not get the doctors for Dermatologue et venerologue in Nantes
Could not get the doctors 

We can see that there are no disponibilites for any doctor. This means that we will have to dig for more doctors as we are only taking the first ones at the moment.

In [36]:
#Now we need to find the fees for each doctor

connector = Driver()
connector.connect("https://www.doctolib.fr")

doctors = pd.read_csv('doctors.csv')
doctors.pop(doctors.columns[0])


for i in range(len(doctors)):
    print(doctors['Name'][i])
    fee = connector.getFees(doctors['Name'][i])
    doctors['Fees'][i] = fee
    print(i, fee)


/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_3132/2052145444.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr
Centre Médical de Soins Immédiats 86


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="logo-doctolib logo-doctolib-white"]"}
  (Session info: chrome=109.0.5414.87)
Stacktrace:
0   chromedriver                        0x0000000102b0b0fc chromedriver + 4223228
1   chromedriver                        0x0000000102a93284 chromedriver + 3732100
2   chromedriver                        0x00000001027475c8 chromedriver + 275912
3   chromedriver                        0x00000001027831b8 chromedriver + 520632
4   chromedriver                        0x00000001027bb360 chromedriver + 750432
5   chromedriver                        0x0000000102775748 chromedriver + 464712
6   chromedriver                        0x00000001027767f0 chromedriver + 468976
7   chromedriver                        0x0000000102adbe08 chromedriver + 4029960
8   chromedriver                        0x0000000102adf698 chromedriver + 4044440
9   chromedriver                        0x0000000102adfcc0

In [ ]:
doctors.to_csv("doctors_fees.csv")

In [ ]:
#Now we need to find the fees for each doctor

connector = Driver()
connector.connect("https://www.doctolib.fr")

doctors = pd.read_csv('doctors_new.csv')
doctors.pop(doctors.columns[0])


for i in range(len(doctors)):
    print(doctors['Name'][i])
    fee = connector.getFees(doctors['Name'][i])
    doctors['Fees'][i] = fee
    print(i, fee)


In [ ]:
doctors.to_csv("doctors_new_fees.csv")